In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from functools import partial
from math import log, sqrt, exp, isclose
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm, t
import scipy
import numpy as np
from scipy.stats import norm
from datetime import datetime
import inspect
import math
import sys
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime
import inspect

In [3]:
ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')
# transfer percentage to value
data = ff.join(mom, how='right') / 100
data

,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,
1926-11-03,0.0020,-0.0020,-0.0033,0.00013,0.0056
1926-11-04,0.0059,-0.0012,0.0065,0.00013,-0.0050
1926-11-05,0.0007,-0.0011,0.0026,0.00013,0.0117
1926-11-06,0.0016,-0.0029,0.0005,0.00013,-0.0003
1926-11-08,0.0052,-0.0012,0.0018,0.00013,-0.0001
...,...,...,...,...,...
2023-01-25,0.0000,-0.0004,0.0065,0.00017,0.0014
2023-01-26,0.0108,-0.0058,0.0001,0.00017,-0.0123
2023-01-27,0.0036,0.0062,-0.0116,0.00017,-0.0246


In [5]:
import math
import sys
import pandas as pd
import numpy as np

def calculate_returns(data, method="discrete", date_col="Date"):
    assert date_col in data.columns

    price_vars = list(data.columns)
    price_vars.remove(date_col)
    price_data = data[price_vars]

    num_vars = len(price_vars)
    num_rows = len(price_data) - 1

    returns_data = np.zeros(shape=(num_rows, num_vars))
    returns_df = pd.DataFrame(returns_data)

    for i in range(num_rows):
        for j in range(num_vars):
            returns_df.iloc[i, j] = price_data.iloc[i + 1, j] / price_data.iloc[i, j]
            if method == "discrete":
                returns_df.iloc[i, j] -= 1
            elif method == "log":
                returns_df.iloc[i, j] = math.log(returns_df.iloc[i, j])
            else:
                sys.exit(1)

    date_series = data[date_col].drop(index=0)
    date_series.index -= 1

    output = pd.DataFrame({date_col: date_series})

    for i in range(num_vars):
        output[price_vars[i]] = returns_df.iloc[:, i]

    return output

In [6]:
all_prices = pd.read_csv("DailyPrices.csv", parse_dates=["Date"])
all_returns = calculate_returns(all_prices, method="discrete", date_col="Date")

/var/folders/2n/lm0km0mn4zlf79tvv2679h740000gn/T/ipykernel_51833/2344943484.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output[price_vars[i]] = returns_df.iloc[:, i]


In [7]:
all_returns = all_returns.set_index("Date")

stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'RF']
dataset = all_returns[stocks].join(data)
dataset

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,...,DIS,GOOGL,JNJ,BAC,CSCO,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-15,0.023152,0.015158,0.008073,0.019724,0.018542,0.091812,0.004836,-0.000201,0.008658,0.006109,...,0.025655,0.007987,0.010326,0.007803,0.020496,0.0187,0.0133,-0.0142,0.0,-0.0091
2022-02-16,-0.001389,-0.020181,0.003806,0.003643,-0.001167,0.000604,-0.008974,-0.002209,0.010159,-0.001739,...,0.010535,0.008268,-0.000598,-0.002302,-0.000369,-0.0002,-0.0009,0.0031,0.0,0.0064
2022-02-17,-0.021269,-0.040778,-0.020227,-0.024103,-0.029282,-0.075591,-0.006141,-0.015700,-0.021809,-0.006653,...,-0.021746,-0.037746,-0.006100,-0.033767,0.028018,-0.0228,-0.0028,0.0110,0.0,0.0103
2022-02-18,-0.009356,-0.007462,-0.005379,-0.010035,-0.009631,-0.035296,-0.003075,-0.007566,-0.013262,0.003987,...,-0.010396,-0.016116,-0.010719,-0.002388,0.025820,-0.0087,-0.0009,0.0093,0.0,0.0104
2022-02-22,-0.017812,-0.019790,-0.011329,-0.004487,-0.000729,-0.010659,-0.088506,-0.020606,-0.015753,-0.002033,...,-0.021604,-0.004521,-0.013590,-0.008703,-0.015906,-0.0118,-0.0048,0.0011,0.0,0.0043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-03,0.024400,-0.011866,0.002527,-0.002747,-0.023621,-0.028053,-0.024397,-0.006315,-0.084315,-0.010742,...,-0.022083,-0.027474,-0.005618,0.008303,-0.013990,NaN,NaN,NaN,NaN,NaN
2023-02-06,-0.017929,-0.002520,0.006822,-0.002514,-0.006116,-0.000521,-0.006727,-0.006809,-0.011703,-0.000259,...,-0.007587,-0.017942,-0.007594,-0.001647,-0.021797,NaN,NaN,NaN,NaN,NaN
2023-02-07,0.019245,0.029883,0.003451,0.005604,0.042022,0.051401,-0.012361,-0.003885,-0.000685,0.014720,...,0.016019,0.046064,0.000245,0.010998,0.005676,NaN,NaN,NaN,NaN,NaN


In [8]:
dataset = dataset.drop(dataset['2023-02'].index)
# calculate arithmetic E(r) in past 10 years
avg_factor_rets = data.loc['2013-1-31':'2023-1-31'].mean(axis=0)
dataset['RF']

/var/folders/2n/lm0km0mn4zlf79tvv2679h740000gn/T/ipykernel_51833/2090827049.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  dataset = dataset.drop(dataset['2023-02'].index)


Date
2022-02-15    0.00000
2022-02-16    0.00000
2022-02-17    0.00000
2022-02-18    0.00000
2022-02-22    0.00000
               ...   
2023-01-25    0.00017
2023-01-26    0.00017
2023-01-27    0.00017
2023-01-30    0.00017
2023-01-31    0.00017
Name: RF, Length: 241, dtype: float64

In [9]:
avg_daily_rets = pd.Series()
for stock in stocks:
  # calculate betas
  model = sm.OLS(dataset[stock] - dataset['RF'], sm.add_constant(dataset[factors]))
  results = model.fit()
    
  # assume alpha = 0
  avg_daily_rets[stock] = (results.params[factors] * avg_factor_rets[factors]).sum() \
                          + avg_factor_rets['RF'] 
# geometric annual returns: mean and covariance
geo_means = np.log(1 + avg_daily_rets) * 255  
geo_covariance = np.log(1 + all_returns[stocks]).cov() * 255
print(geo_means)

/var/folders/2n/lm0km0mn4zlf79tvv2679h740000gn/T/ipykernel_51833/369180229.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  avg_daily_rets = pd.Series()


AAPL     0.093860
META     0.361449
UNH     -0.069173
MA       0.191821
MSFT     0.111293
NVDA     0.464330
HD       0.199479
PFE     -0.029811
AMZN     0.151040
BRK-B    0.111360
PG       0.111582
XOM      0.043400
TSLA    -0.104159
JPM      0.300436
V        0.189308
DIS      0.309686
GOOGL    0.151795
JNJ     -0.054679
BAC      0.264885
CSCO     0.189703
dtype: float64


In [11]:
# arithmetic annual returns: mean and covariance
arith_means = np.exp(geo_means + np.diagonal(geo_covariance.values) / 2) - 1

nstocks = geo_covariance.shape[0]
arith_covariance = np.empty((nstocks, nstocks), dtype=float)
for i in range(nstocks):
  for j in range(i, nstocks):
    mu_i, mu_j = geo_means.iloc[i], geo_means.iloc[j]
    sigma2_i, sigma2_j = geo_covariance.iloc[i, i], geo_covariance.iloc[j, j]
    sigma_ij = geo_covariance.iloc[i, j]
    arith_covariance[i, j] = np.exp(mu_i + mu_j + (sigma2_i + sigma2_j) / 2) * (np.exp(sigma_ij) - 1)
    arith_covariance[j, i] = arith_covariance[i, j]
arith_covariance = pd.DataFrame(arith_covariance, columns=stocks, index=stocks)

print(arith_means)
arith_covariance

AAPL     0.171146
META     0.762903
UNH     -0.037641
MA       0.271602
MSFT     0.192290
NVDA     0.950026
HD       0.282706
PFE      0.005805
AMZN     0.316488
BRK-B    0.146426
PG       0.145970
XOM      0.110124
TSLA     0.139654
JPM      0.407970
V        0.260227
DIS      0.462612
GOOGL    0.262884
JNJ     -0.038059
BAC      0.371124
CSCO     0.263864
dtype: float64


,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.187678,0.312484,0.043679,0.127405,0.152881,0.430491,0.104033,0.039731,0.203032,0.077363,0.051068,0.050553,0.228246,0.100238,0.110482,0.159249,0.176847,0.026298,0.110812,0.102854
META,0.312484,1.579908,0.031448,0.243338,0.322789,0.931842,0.233174,0.083442,0.503301,0.129622,0.069139,0.042602,0.382796,0.192019,0.198846,0.351335,0.446682,0.036992,0.225918,0.177735
UNH,0.043679,0.031448,0.058855,0.039281,0.043127,0.091133,0.033209,0.031809,0.046285,0.031884,0.031573,0.029309,0.044878,0.046263,0.037018,0.032889,0.037246,0.021686,0.046853,0.035854
MA,0.127405,0.243338,0.039281,0.164553,0.127062,0.368959,0.096262,0.044228,0.172149,0.071805,0.046433,0.045088,0.151263,0.108628,0.139296,0.151763,0.133338,0.021706,0.115522,0.086362
MSFT,0.152881,0.322789,0.043127,0.127062,0.196004,0.451145,0.113558,0.043527,0.228225,0.074625,0.047474,0.042509,0.193638,0.098659,0.106985,0.162759,0.194404,0.023858,0.111051,0.095565
NVDA,0.430491,0.931842,0.091133,0.368959,0.451145,1.910206,0.298844,0.094890,0.645605,0.198885,0.096143,0.124574,0.765912,0.286210,0.311906,0.495162,0.512562,0.042113,0.324821,0.257123
HD,0.104033,0.233174,0.033209,0.096262,0.113558,0.298844,0.171198,0.044521,0.174285,0.064209,0.052242,0.023037,0.120132,0.081309,0.084076,0.125715,0.117765,0.028407,0.084807,0.081559
PFE,0.039731,0.083442,0.031809,0.044228,0.043527,0.094890,0.044521,0.074653,0.051819,0.036963,0.032475,0.022817,0.026637,0.046317,0.040044,0.037424,0.038315,0.027375,0.043697,0.038455
AMZN,0.203032,0.503301,0.046285,0.172149,0.228225,0.645605,0.174285,0.051819,0.487487,0.104121,0.047306,0.056700,0.313940,0.137938,0.147164,0.259931,0.272511,0.029698,0.162978,0.125555
BRK-B,0.077363,0.129622,0.031884,0.071805,0.074625,0.198885,0.064209,0.036963,0.104121,0.068186,0.033367,0.044392,0.084380,0.078316,0.062826,0.089518,0.084616,0.021634,0.082524,0.060501


In [13]:
# calculate the most efficient portfolio which has the highest Sharpe ratio 
def neg_sharpe_ratio(weights, mean, cov, r):
  returns = mean @ weights.T
  std = np.sqrt(weights @ cov @ weights.T)
  return -(returns - r) / std

args = (arith_means, arith_covariance, 0.0025)
bounds = [(0.0, 1) for _ in stocks]
x0 = np.array(nstocks*[1 / nstocks])
constraints = {'type':'eq', 'fun': lambda x: np.sum(x) - 1}
results = scipy.optimize.minimize(neg_sharpe_ratio, x0=x0, args=args, bounds=bounds, constraints=constraints)
opt_sharpe, opt_weights = -results.fun, pd.Series(results.x, index=stocks)
opt_weights = pd.DataFrame(opt_weights, columns=['weights(%)'])
opt_weights['weights(%)'] = round(opt_weights*100, 2)

print("The most efficient portfolio consists of: ")
print(opt_weights)
print("The Portfolio's Sharpe Ratio is: " + str(opt_sharpe))

The most efficient portfolio consists of: 
       weights(%)
AAPL         0.00
META         4.27
UNH          0.00
MA           0.00
MSFT         0.00
NVDA         0.63
HD           7.76
PFE          0.00
AMZN         0.00
BRK-B        0.00
PG           8.35
XOM          0.00
TSLA         0.00
JPM         58.01
V            0.00
DIS         15.91
GOOGL        0.00
JNJ          0.00
BAC          0.00
CSCO         5.07
The Portfolio's Sharpe Ratio is: 1.0462140296202906
